# Backtesting ML Regression-Based

In [ ]:
???

Classification Strategy | Regression Strategy
-|-
![](src/res_classification.png) | ![](src/res_regression.png)

## Load the model

In [1]:
import pickle

In [3]:
with open("models/model_dt_regression.pkl", 'rb') as f:
    model_dt = pickle.load(f)

## Load the data

In [4]:
import pandas as pd

df = pd.read_excel('data/Microsoft_LinkedIn_Processed.xlsx', index_col=0, parse_dates=['Date'])
df

,Open,High,Low,Close,Adj Close,Volume,change_tomorrow,change_tomorrow_direction
Date,,,,,,,,
2016-12-08,61.299999,61.580002,60.840000,61.009998,55.705235,21220800,1.549141,UP
2016-12-09,61.180000,61.990002,61.130001,61.970001,56.581772,27349400,0.321694,UP
2016-12-12,61.820000,62.299999,61.720001,62.169998,56.764374,20198100,1.286125,UP
2016-12-13,62.500000,63.419998,62.240002,62.980000,57.503944,35718900,-0.478620,DOWN
2016-12-14,63.000000,63.450001,62.529999,62.680000,57.230022,30352700,-0.159793,DOWN
...,...,...,...,...,...,...,...,...
2023-12-18,369.450012,373.000000,368.679993,372.649994,372.649994,21802900,0.163429,UP
2023-12-19,371.489990,373.260010,369.839996,373.260010,373.260010,20603700,-0.712324,DOWN
2023-12-20,375.000000,376.029999,370.529999,370.619995,370.619995,26316700,0.781714,UP


## Backtesting.py Library

### Create your Strategy Class

In [5]:
from backtesting import Backtest, Strategy

In [6]:
class Regression(Strategy):
  def init(self):
    self.model = model_dt
    self.already_bought = False

  def next(self):
      explanatory_today = self.data.df.iloc[[-1],:]
      forecast_tomorrow = model_dt.predict(explanatory_today)[0]
      
      # conditions to sell or buy
      if forecast_tomorrow > 5 and self.already_bought==False:
          self.buy()
          self.already_bought = True

      elif forecast_tomorrow < -5 and self.already_bought:
          self.sell()
          self.already_bought = False

      else:
          pass

### Define initial conditions

In [11]:
df_explanatory = df.drop(columns=['Adj Close', 'change_tomorrow', 'change_tomorrow_direction'])

In [12]:
bt = Backtest(df_explanatory, Regression,
              cash=10000, commission=.002, exclusive_orders=True)

### Run backtesting

In [13]:
results = bt.run()

### Interpret backtesting results

In [14]:
results.to_frame(name='Values').loc[:'Return [%]']

,Values
Start,2016-12-08 00:00:00
End,2023-12-22 00:00:00
Duration,2570 days 00:00:00
Exposure Time [%],87.41535
Equity Final [$],43811.377117
Equity Peak [$],44866.719403
Return [%],338.113771


### Visualize the strategy performance

In [15]:
bt.plot(filename='reports_backtesting/backtesting_regression.html')

Row(id='1407', ...)